# <span style="color:teal">RBFE Network - Analysis</span>


In [ ]:
# import libraries

from scipy.stats import sem as sem
import sys
import glob

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "mcl1"
main_dir = f"{bench_folder}/extracted/{protein}"
# main_dir = f"/backup/GROMACS_reruns/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_combined.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"
results_folder = f"{main_dir}/outputs_extracted/results"
output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)

In [ ]:
all_analysis_object = analysis_network(
    results_folder,
    exp_file=exp_file,
    net_file=net_file,
    output_folder=output_folder,
    analysis_prot=ana_file,
    # method = "updated",
    # engines = "GROMACS"
)

# can add any other results files
all_analysis_object.compute_results()

In [ ]:
all_analysis_object.add_ligands_folder(f"/home/anna/Documents/benchmark/inputs/{protein}/ligands")
all_analysis_object._initialise_graph_object()
all_analysis_object.draw_graph()

In [ ]:
# check if there are any failed perturbations
failed_perts = all_analysis_object.failed_runs("GROMACS")

for pert in sorted(failed_perts):
    print(pert)

all_analysis_object.draw_failed_perturbations()

In [ ]:
all_analysis_object.disconnected_ligands(eng="GROMACS")

In [ ]:
# can compute convergence for all
all_analysis_object.compute_convergence(main_dir=main_dir)
all_analysis_object.plot_convergence()

There are different options for plotting:

bar (pert or lig)

scatter (pert or lig) - can be plotted using cinnabar

eng vs eng

other results (eg from other workflows)

outliers

histograms (for the error of the runs, sem, legs)

In [ ]:
# bar
all_analysis_object.plot_bar_lig()
all_analysis_object.plot_bar_pert()

# scatter
all_analysis_object.plot_scatter_lig()
all_analysis_object.plot_scatter_pert()
all_analysis_object.plot_scatter_lig(use_cinnabar=True)
all_analysis_object.plot_scatter_pert(use_cinnabar=True)

for eng in all_analysis_object.engines:
    all_analysis_object.plot_scatter_lig(engine=eng)
    all_analysis_object.plot_scatter_pert(engine=eng)

    # outliers
    all_analysis_object.plot_outliers(engine=eng)
    all_analysis_object.plot_outliers(engine=eng, pert_val="val")

all_analysis_object.plot_histogram_legs()
all_analysis_object.plot_histogram_repeats()
all_analysis_object.plot_histogram_sem(pert_val="pert")
all_analysis_object.plot_histogram_sem(pert_val="val")

calculate the stats:

In [ ]:
df = all_analysis_object.calc_mad_engines(pert_val="pert")
# all_analysis_object.calc_mad_engines(pert_val="val")
df

In [ ]:
df = all_analysis_object.calc_mae_engines(pert_val="pert")
# all_analysis_object.calc_mad_engines(pert_val="val")
df

sort by binding affinity

In [ ]:
all_analysis_object.sort_ligands_by_binding_affinity(engine="GROMACS")
all_analysis_object.sort_ligands_by_experimental_binding_affinity()

plot other results

In [ ]:
eng = "GROMACS"
rerun_folder = "/backup/GROMACS_reruns/tyk2/outputs_extracted/results"

other_results = glob.glob(
    f"{rerun_folder}/freenrg_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)
bound_results = glob.glob(
    f"{rerun_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)
free_results = glob.glob(
    f"{rerun_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)
all_analysis_object.compute_other_results(
    other_results, name=f"GROMACS_reruns", bound_files=bound_results, free_files=free_results
)


In [ ]:
# remove any non main netowrk perturbations
for eng in all_analysis_object.other_results_names:
    for pert in all_analysis_object._perturbations_dict[eng]:
        if pert not in all_analysis_object._perturbations_dict["GROMACS"]:
            all_analysis_object.remove_perturbations(pert, name=eng)

In [ ]:
all_analysis_object.plot_outliers(engines=["GROMACS","GROMACS_reruns"], no_outliers=3)

In [ ]:
all_analysis_object.plot_histogram_repeats(engines=["GROMACS","GROMACS_reruns"])

remove a perturbation

In [ ]:
# check which perturbations are outliers
all_analysis_object.plot_outliers(engine="GROMACS", outliers=5, pert_val="pert")
# all_analysis_object._plotting_object.outlier(engines="AMBER", outliers=5, name="Exscientia")

In [ ]:
# remove perturbation for all engines
all_analysis_object.remove_perturbations("lig_c~lig_d")

remove outliers

In [ ]:
all_analysis_object.remove_outliers(threshold=10, verbose=True)

# can then recalculate the above.